In [68]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

df_hypopt       = df
split_date      = '2019-01-01 00:00:00'
df_hypopt_train = df_hypopt.loc[df_hypopt.Datetime < split_date].copy()
df_hypopt_test  = df_hypopt.loc[df_hypopt.Datetime >= split_date].copy()

y_train         = df_hypopt_train['AIL']
y_test          = df_hypopt_test['AIL']
X_train         = df_hypopt_train.drop(['Datetime', 'AIL','just_date'], axis = 1)
X_test          = df_hypopt_test.drop (['Datetime', 'AIL','just_date'], axis = 1)


xgb_model = XGBRegressor(n_jobs=1)
clf       = GridSearchCV(xgb_model,
                   {'max_depth': [2, 4, 6, 8, 10, 12],
                   'n_estimators': [50, 100, 200, 500, 1000]}, 
                    verbose=1, n_jobs=1)
clf.fit(X_train.values, y_train.values)
print(clf.best_score_)
print(clf.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 41.3min finished


0.9847942434439814
{'max_depth': 2, 'n_estimators': 1000}


In [ ]:
y_train = pd.DataFrame(df, columns = ['AIL'])
y_train = y_train.values
X_train = pd.DataFrame(df, columns = loaded_list)
X_train = X_train.values
#model with optimized hyperparameter
model=XGBRegressor(colsample_bylevel = 1,
                   colsample_bynode  = 1, 
                   colsample_bytree  = 0.7936909313888836, 
                   gamma             = 0.7977155727005966, 
                   importance_type   = 'gain', 
                   learning_rate     = 0.1, 
                   max_delta_step    = 0,
                   max_depth         = 11, 
                   min_child_weight  = 3.0, 
                   missing           = None, 
                   n_estimators      = 100,
                   n_jobs            = 1, 
                   nthread           = None, 
                   objective         ='reg:squarederror',
                   random_state      = 0, 
                   reg_alpha         = 1, 
                   reg_lambda        = 0.8521680716712875, 
                   scale_pos_weight  = 1,
                   seed              = None, 
                   silent            = None, 
                   subsample         = 1, 
                   verbosity         = 1)

model.fit(X_train,y_train)


import joblib

from joblib import dump
# save model to file
model_name = 'model_generated_on_{}_{}_{}.joblib.dat'.format(datetime.now().day, datetime.now().month, datetime.now().year)
joblib.dump(model, filename = model_name)

In [61]:
# to build the model
from sklearn.linear_model import Lasso

# to evaluate the model
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# set up the model
# remember to set the random_state / seed

df_hypopt       = df
split_date      = '2020-01-01 00:00:00'
df_hypopt_train = df_hypopt.loc[df_hypopt.Datetime < split_date].copy()
df_hypopt_test  = df_hypopt.loc[df_hypopt.Datetime >= split_date].copy()

y_train         = df_hypopt_train['AIL']
y_test          = df_hypopt_test['AIL']
X_train         = df_hypopt_train.drop(['Datetime', 'AIL','just_date'], axis = 1)
X_test          = df_hypopt_test.drop (['Datetime', 'AIL','just_date'], axis = 1)



lin_model = Lasso(alpha=0.001, random_state=0, max_iter = 100000)

# train the model

lin_model.fit(X_train.values, y_train.values)

# evaluate the model:
# ====================

# remember that we log transformed the output (SalePrice)
# in our feature engineering notebook (step 2).

# In order to get the true performance of the Lasso
# we need to transform both the target and the predictions
# back to the original house prices values.

# We will evaluate performance using the mean squared error and
# the root of the mean squared error and r2

# make predictions for train set
pred = lin_model.predict(X_train)

# determine mse and rmse
print('train mse: {}' . format(mean_squared_error(y_train.values, pred)))
print('train rmse: {}'. format(sqrt(mean_squared_error(y_train.values,pred))))
print('train r2: {}'  . format(r2_score(y_train.values, pred)))
print()

# make predictions for test set
pred = lin_model.predict(X_test.values)

print('train mse: {}' . format(mean_squared_error(y_test.values, pred)))
print('train rmse: {}'. format(sqrt(mean_squared_error(y_test.values,pred))))
print('train r2: {}'  . format(r2_score(y_test.values, pred)))
print()

train mse: 3632.2917992626753
train rmse: 60.268497569316224
train r2: 0.9924881763713345

train mse: 4136.804203160197
train rmse: 64.3179928415074
train r2: 0.9824384153735127



/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6157506.957534292, tolerance: 163631.04428862297
  positive)


In [70]:
# use automatically configured the lasso regression algorithm
from numpy import arange
from pandas import read_csv
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=0)
# define model
model = LassoCV(alphas=arange(0, 1, 0.1), cv=cv, n_jobs=-1, random_state = 0, tol = 0.001, max_iter = 100000)
# fit model
model.fit(X_train.values, y_train.values)
# summarize chosen configuration
print('alpha: %f' % model.alpha_)

/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1661942.692916287, tolerance: 1059518.6171118394
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4492732.066624641, tolerance: 1062276.4513904138
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2587873.417096679, tolerance: 1064551.4438648093
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/

/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4472917.079417706, tolerance: 1068391.625180328
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2223606.4491302557, tolerance: 1062131.5242662842
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4359646.19768095, tolerance: 1063938.5521941828
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/s

/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4478064.697810531, tolerance: 1059518.6171118394
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1830381.246269036, tolerance: 1062131.5242662842
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: Conv

/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4399074.789141178, toler

/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1107105.7462670002, tolerance: 1059979.2891438524
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to incr

alpha: 0.000000


/Users/abulhasanfahad/opt/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4993267.321890177, tolerance: 1183126.504830383
  positive)


In [21]:
#split_date = '2019-01-01 00:00:00'

#df_train = df.loc[df.Datetime < split_date].copy()
#df_test  = df.loc[df.Datetime >= split_date].copy()